<p style="text-align: center; font-size:20px">Case Stone Payments
<p style="text-align: center; font-size:15px">Customer Relationship Management (CRM) Analytics
<hr />

# Cleaning and Features Extracting

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Loading df's
creds = pd.read_csv("creds.csv")
cases = pd.read_csv("cases.csv")

<hr/>

# > First glance

### creds.csv

| Coluna                  | Descrição     |
|:----------------------- |-------------|
|cred_date                |Accreditation date|
|shipping_address_city    |Delivery address – city|
|shipping_address_state   |Delivery address - state|
|max_machine              |Purchased product|
|accountid                |Table secondary key|

In [3]:
creds.head()

,Unnamed: 0,cred_date,shipping_address_city,shipping_address_state,max_machine,accountid
0,0,2020-04-18,Feira de Santana,BA,T1,NaN
1,1,2020-10-16,Bacuri,MA,T1,NaN
2,2,2020-09-01,Bernardo Sayão,TO,T1,NaN
3,3,2020-08-29,Rio de Janeiro,RJ,T3,NaN
4,4,2020-07-28,São Gonçalo,RJ,T3,0013j00002z0CeEAAU


In [4]:
# to understand general dataframe information
creds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126989 entries, 0 to 126988
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Unnamed: 0              126989 non-null  int64 
 1   cred_date               126989 non-null  object
 2   shipping_address_city   126989 non-null  object
 3   shipping_address_state  126989 non-null  object
 4   max_machine             126989 non-null  object
 5   accountid               77489 non-null   object
dtypes: int64(1), object(5)
memory usage: 5.8+ MB


In [5]:
# Check for unique values of columns that appear to be categorical
pd.unique(creds[['max_machine']].values.ravel('K'))

array(['T1', 'T3', 'T2', 'NONE'], dtype=object)

### Notes for treatment
- Missing values (only in *accountid* column)
- Convert datatype from *cred_date* to datetime64
- Remove Unnamed column (Reason: it's the same as index *and doesn't let you know if there is duplicate data*)
- Remove duplicate lines, if any, leaving only the first occurrence

<hr/>

## cases.csv

| Coluna                  | Descrição     |
|:----------------------- |-------------|
|accountid                |Table primary key|
|date_ref                 |Call date|
|channelid                |Call channel (chat = 1; phone = 2; email = 3)|
|waitingtime              |Waiting time for service (seconds)|
|missed                   |Answered Call flag|
|pesquisa_de_satisfa_o__c |Satisfaction Survey Completed flag|
|assunto                  |Call subject|
|id                       |Table key|

In [6]:
cases.head()

,Unnamed: 0,accountid,date_ref,channelid,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto,Id
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU


In [7]:
# To understand general dataframe information
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126989 entries, 0 to 126988
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                126989 non-null  int64  
 1   accountid                 77489 non-null   object 
 2   date_ref                  77489 non-null   object 
 3   channelid                 77489 non-null   float64
 4   waitingtime               77489 non-null   float64
 5   missed                    77489 non-null   object 
 6   pesquisa_de_satisfa_o__c  11585 non-null   object 
 7   assunto                   77489 non-null   object 
 8   Id                        77489 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 8.7+ MB


In [8]:
# Check for unique values of columns that appear to be categorical
pd.unique(cases[['channelid', 'missed', 'pesquisa_de_satisfa_o__c']].values.ravel('K'))

array([nan, 2.0, False, True, 'Enviado', 'Incrivel', 'Normal', 'Bom',
       'NaoResolveu', 'Triste', 'Cansativo', 'MalAtendido', 'Feliz',
       'Pessimo'], dtype=object)

### Notes for treatment
- There are lines where there are no values in all columns (NaN)
- There are lines in which there are missing values only in the column *pesquisa_de_satisfac_o__c*
- Convert *date_ref* datatype to datetime64
- Convert the datatype from *pesquisa_de_satisfac_o__c* to a categorical variable, if you apply ML
- Remove *Unnamed* column (same as index *and does not let you know if there is duplicate data*)
- Remove *Id* column (same value as accountid)
- Remove *channelid* column -> There is only 1 value: "2" (phone)
- Remove duplicate lines, if any, leaving only the first occurrence
- Modify the column title *pesquisa_de_satisfa_o__c*, to facilitate analysis

QUESTION:
- Are all _date_ref_ later than _cred_date_?

<hr/>

# > Cleaning

### Creds

In [9]:
# Remove missing values (only in *accountid* column)
creds = creds.dropna(how='any',axis=0)

In [10]:
# Remove *Unnamed* column (Reason: same as index and does not let you know if there is duplicate data)
creds.drop(['Unnamed: 0'], axis=1, inplace=True)

The decision to remove rows where there are no *accountid* values comes from the fact that the case questions don't use the other columns as variables at all. If there was a need to correlate *cred_date* with *max_machine*, for example, I would choose to keep the lines with *accountid* blank. However, the main value will be taken from the join of the two dataframes through *accountid*

In [11]:
# Convert *cred_date* datatype to datetime64
creds.cred_date = pd.to_datetime(creds.cred_date)

In [12]:
# Check for duplicate cases
# It is only possible to check after deleting the *Unnamed* column, which artificially differentiates occurrences
creds_duplicate = creds[creds.duplicated(keep='first')]
creds_duplicate.head()

,cred_date,shipping_address_city,shipping_address_state,max_machine,accountid
5,2020-07-28,São Gonçalo,RJ,T3,0013j00002z0CeEAAU
6,2020-07-28,São Gonçalo,RJ,T3,0013j00002z0CeEAAU
8,2020-08-28,Itaboraí,RJ,T1,0013j00002zQgldAAC
9,2020-08-28,Itaboraí,RJ,T1,0013j00002zQgldAAC
10,2020-08-28,Itaboraí,RJ,T1,0013j00002zQgldAAC


**Há 46060 casos duplicados**

In [13]:
# Remove duplicate lines but keep the first occurrence
creds = creds.drop_duplicates(keep='first')

In [14]:
creds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31429 entries, 4 to 126982
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   cred_date               31429 non-null  datetime64[ns]
 1   shipping_address_city   31429 non-null  object        
 2   shipping_address_state  31429 non-null  object        
 3   max_machine             31429 non-null  object        
 4   accountid               31429 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.4+ MB


In [15]:
creds.head()

,cred_date,shipping_address_city,shipping_address_state,max_machine,accountid
4,2020-07-28,São Gonçalo,RJ,T3,0013j00002z0CeEAAU
7,2020-08-28,Itaboraí,RJ,T1,0013j00002zQgldAAC
15,2020-10-02,Joinville,SC,T1,0013j0000304oN9AAI
23,2020-07-24,São João de Meriti,RJ,T2,0013j00002yzpxZAAQ
25,2020-07-29,Nortelândia,MT,T1,0013j00002z0RA5AAM


### Cases

In [16]:
# Remove 'Unnamed', 'Id' and 'channelid' column
cases.drop(['Unnamed: 0', 'Id', 'channelid'], axis=1, inplace=True)

In [17]:
# Check for duplicate cases
# It is only possible to check after deleting the Unnamed column, which artificially differentiates occurrences
cases_duplicated = cases[cases.duplicated(keep='first')]
print(cases_duplicated)

       accountid date_ref  waitingtime missed pesquisa_de_satisfa_o__c assunto
1            NaN      NaN          NaN    NaN                      NaN     NaN
2            NaN      NaN          NaN    NaN                      NaN     NaN
3            NaN      NaN          NaN    NaN                      NaN     NaN
11           NaN      NaN          NaN    NaN                      NaN     NaN
12           NaN      NaN          NaN    NaN                      NaN     NaN
...          ...      ...          ...    ...                      ...     ...
126946       NaN      NaN          NaN    NaN                      NaN     NaN
126960       NaN      NaN          NaN    NaN                      NaN     NaN
126967       NaN      NaN          NaN    NaN                      NaN     NaN
126973       NaN      NaN          NaN    NaN                      NaN     NaN
126981       NaN      NaN          NaN    NaN                      NaN     NaN

[49499 rows x 6 columns]


Apparently there are rows where all columns are empty (*NaN*) - let's make sure:

In [18]:
cases_duplicated.describe(include="all")

,accountid,date_ref,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto
count,0,0,0.0,0,0,0
unique,0,0,NaN,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN


As noted, **there are no duplicate cases, but there are 49499 rows where there are no values (*NaN*) in all columns**

I decided to **remove** those rows where all columns are empty - however I decided to **not to remove** the rows where only the *satisfy_the_c* column is empty, as the total df rows would be reduced from 77489 to 11585 , losing a lot of information about other important variables that may have their correlations studied, such as e.g. *Subject* vs. *Waiting time*.

In [19]:
# Checking before how many lines the df has, even with null values
cases.shape

(126989, 6)

In [20]:
# Remove only rows where all columns ('all') have null values
cases = cases.dropna(how='all')

In [21]:
# Checking the change
cases.shape

(77489, 6)

In [22]:
# Modify the title of the pesquisa_de_satisfa_o__c column, to facilitate analysis
cases.rename(columns={'pesquisa_de_satisfa_o__c': 'pesquisa_sat'}, inplace=True)

In [23]:
# Convert *date_ref* datatype to datetime64
cases.date_ref = pd.to_datetime(cases.date_ref)

In [24]:
# Replace null values for "SemValor" (NoValue) in *sat_search* column
# To be able to convert all column values to categorical variables
cases['pesquisa_sat'].fillna('SemValor', inplace= True)

In [25]:
# Convert datatype from *pesquisa_sat* to categorical variable
pesquisa_cat = cases['pesquisa_sat'].unique().tolist()
cases['pesquisa_sat'] = pd.Categorical(cases['pesquisa_sat'], categories=pesquisa_cat, ordered=False)

In [26]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77489 entries, 4 to 126988
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   accountid     77489 non-null  object        
 1   date_ref      77489 non-null  datetime64[ns]
 2   waitingtime   77489 non-null  float64       
 3   missed        77489 non-null  object        
 4   pesquisa_sat  77489 non-null  category      
 5   assunto       77489 non-null  object        
dtypes: category(1), datetime64[ns](1), float64(1), object(3)
memory usage: 3.6+ MB


In [27]:
cases.head()

,accountid,date_ref,waitingtime,missed,pesquisa_sat,assunto
4,0013j00002z0CeEAAU,2020-07-31,15.0,False,SemValor,Aplicativo:Dúvidas funcionalidades App:Primeir...
5,0013j00002z0CeEAAU,2020-07-31,15.0,False,SemValor,Produto:S920:Ativação
6,0013j00002z0CeEAAU,2020-09-23,1.0,False,SemValor,Produto:Cartão pré-pago:Dúvidas
7,0013j00002zQgldAAC,2020-08-29,6.0,False,SemValor,Logística:Envio de Cartão:Tracking
8,0013j00002zQgldAAC,2020-08-29,6.0,False,SemValor,Pedido:Cancelamento:Outros


# > Features extraction / selection

In [28]:
# inner join of *creds* and *cases*
df = cases.join(creds.set_index('accountid'), on='accountid')
df.head()

,accountid,date_ref,waitingtime,missed,pesquisa_sat,assunto,cred_date,shipping_address_city,shipping_address_state,max_machine
4,0013j00002z0CeEAAU,2020-07-31,15.0,False,SemValor,Aplicativo:Dúvidas funcionalidades App:Primeir...,2020-07-28,São Gonçalo,RJ,T3
5,0013j00002z0CeEAAU,2020-07-31,15.0,False,SemValor,Produto:S920:Ativação,2020-07-28,São Gonçalo,RJ,T3
6,0013j00002z0CeEAAU,2020-09-23,1.0,False,SemValor,Produto:Cartão pré-pago:Dúvidas,2020-07-28,São Gonçalo,RJ,T3
7,0013j00002zQgldAAC,2020-08-29,6.0,False,SemValor,Logística:Envio de Cartão:Tracking,2020-08-28,Itaboraí,RJ,T1
8,0013j00002zQgldAAC,2020-08-29,6.0,False,SemValor,Pedido:Cancelamento:Outros,2020-08-28,Itaboraí,RJ,T1


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77489 entries, 4 to 126988
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   accountid               77489 non-null  object        
 1   date_ref                77489 non-null  datetime64[ns]
 2   waitingtime             77489 non-null  float64       
 3   missed                  77489 non-null  object        
 4   pesquisa_sat            77489 non-null  category      
 5   assunto                 77489 non-null  object        
 6   cred_date               77489 non-null  datetime64[ns]
 7   shipping_address_city   77489 non-null  object        
 8   shipping_address_state  77489 non-null  object        
 9   max_machine             77489 non-null  object        
dtypes: category(1), datetime64[ns](2), float64(1), object(6)
memory usage: 6.0+ MB


To correlate with satisfaction surveys and to understand if all *date_ref* values come after *cred_date*: <br>
Feature **tempo_de_casa** (in days)

In [30]:
df['tempo_de_casa'] = ((df['date_ref'] - df['cred_date']) 
                               / (np.timedelta64(1, 'D')))

Seeing if we can make the subject more succinct:

In [31]:
df['assunto'].nunique()

221

There are many unique subjects (221), so I'll just leave the macro group for each subject:

In [32]:
#split the subjects
df['assunto_short'] = df['assunto'].str.rsplit(':').str[-3] 
df['assunto_short'].replace({"":"Outros"}, inplace=True)

In [33]:
# rearranging df
df = df[['accountid', 'cred_date', 'date_ref', 'tempo_de_casa', 'max_machine', 'assunto_short', 'waitingtime', 'pesquisa_sat']]

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77489 entries, 4 to 126988
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   accountid      77489 non-null  object        
 1   cred_date      77489 non-null  datetime64[ns]
 2   date_ref       77489 non-null  datetime64[ns]
 3   tempo_de_casa  77489 non-null  float64       
 4   max_machine    77489 non-null  object        
 5   assunto_short  77489 non-null  object        
 6   waitingtime    77489 non-null  float64       
 7   pesquisa_sat   77489 non-null  category      
dtypes: category(1), datetime64[ns](2), float64(2), object(3)
memory usage: 4.8+ MB


In [35]:
df.head()

,accountid,cred_date,date_ref,tempo_de_casa,max_machine,assunto_short,waitingtime,pesquisa_sat
4,0013j00002z0CeEAAU,2020-07-28,2020-07-31,3.0,T3,Aplicativo,15.0,SemValor
5,0013j00002z0CeEAAU,2020-07-28,2020-07-31,3.0,T3,Produto,15.0,SemValor
6,0013j00002z0CeEAAU,2020-07-28,2020-09-23,57.0,T3,Produto,1.0,SemValor
7,0013j00002zQgldAAC,2020-08-28,2020-08-29,1.0,T1,Logística,6.0,SemValor
8,0013j00002zQgldAAC,2020-08-28,2020-08-29,1.0,T1,Pedido,6.0,SemValor


To find out if there are negative *tempo_de_casa* values:

In [36]:
(df['tempo_de_casa'] < 0).any().any()

True

There are negative values, let's see if the _subject_ helps to understand

In [37]:
df_negatives = df.where(df['tempo_de_casa'] < 0) # separates only negative values and saves to df_negatives
df_negatives = df_negatives.dropna(how='any',axis=0) # delete null tempo_de_casa
df_negatives.head(10)

,accountid,cred_date,date_ref,tempo_de_casa,max_machine,assunto_short,waitingtime,pesquisa_sat
56,0013j00002zg2jKAAQ,2020-09-15,2020-09-14,-1.0,T2,Produto,8.0,SemValor
64,0013j00002u3av5AAA,2020-06-05,2020-06-03,-2.0,T1,Pedido,4.0,Enviado
149,0013j00002ttVPwAAM,2020-06-02,2020-06-01,-1.0,T3,Pedido,11.0,Enviado
199,0013j00002yzcmdAAA,2020-09-01,2020-08-31,-1.0,T1,Produto,7.0,SemValor
205,0013j000030E24pAAC,2020-10-27,2020-10-22,-5.0,T1,Logística,8.0,SemValor
216,0013j00002yeAHuAAM,2020-06-18,2020-06-17,-1.0,T1,Pedido,6.0,Enviado
414,0013j00002z0dSHAAY,2020-07-30,2020-07-29,-1.0,T2,Pedido,43.0,SemValor
459,0013j00002ttSLPAA2,2020-05-06,2020-05-05,-1.0,T1,Pedido,14.0,Enviado
498,0013j00002zgZX2AAM,2020-09-24,2020-09-20,-4.0,T1,Aplicativo,13.0,SemValor
565,0013j00002zQUj0AAG,2020-08-29,2020-08-28,-1.0,T1,Pedido,10.0,SemValor


There are negative values, which indicate that they are not just "complaints", but also initial contacts about the product, as we can see in the subject: Activation, Order Confirmation, etc.

**Saving the file for exploratory analysis**

In [38]:
df.to_csv("df_cleaned.csv", date_format='%Y-%m-%d', index=False)